In [1]:
import sqlite3
import pprint
from pymongo import MongoClient
conn = sqlite3.connect('northwind.db')
c = conn.cursor()

In [2]:
# Select customer from customer table
c.execute("SELECT * FROM Customers WHERE CustomerID = 'ALFKI'")
c.fetchone()

('ALFKI',
 'Alfreds Futterkiste',
 'Maria Anders',
 'Sales Representative',
 'Obere Str. 57',
 'Berlin',
 None,
 '12209',
 'Germany',
 '030-0074321',
 '030-0076545')

In [3]:
# Gets all orders which the customer have made and the product info
c.execute("""SELECT CustomerALFKI.OrderID, CustomerALFKI.CustomerID, pd.ProductID FROM 
            ( SELECT * FROM Orders WHERE CustomerID = 'ALFKI' ) CustomerALFKI 
            INNER JOIN 'Order Details' od
            on CustomerALFKI.OrderID = od.OrderID
            INNER JOIN 'Products' pd
            on od.ProductID = pd.ProductID""")
c.fetchall()

[(10643, 'ALFKI', 28),
 (10643, 'ALFKI', 39),
 (10643, 'ALFKI', 46),
 (10692, 'ALFKI', 63),
 (10702, 'ALFKI', 3),
 (10702, 'ALFKI', 76),
 (10835, 'ALFKI', 59),
 (10835, 'ALFKI', 77),
 (10952, 'ALFKI', 6),
 (10952, 'ALFKI', 28),
 (11011, 'ALFKI', 58),
 (11011, 'ALFKI', 71)]

In [4]:
# Gets all orders which the customer have made
c.execute("""SELECT CustomerALFKI.OrderID, CustomerALFKI.CustomerID, od.ProductID FROM 
            ( SELECT * FROM Orders WHERE CustomerID = 'ALFKI' ) CustomerALFKI 
            INNER JOIN 'Order Details' od
            on CustomerALFKI.OrderID = od.OrderID""")
c.fetchall()

[(10643, 'ALFKI', 28),
 (10643, 'ALFKI', 39),
 (10643, 'ALFKI', 46),
 (10692, 'ALFKI', 63),
 (10702, 'ALFKI', 3),
 (10702, 'ALFKI', 76),
 (10835, 'ALFKI', 59),
 (10835, 'ALFKI', 77),
 (10952, 'ALFKI', 6),
 (10952, 'ALFKI', 28),
 (11011, 'ALFKI', 58),
 (11011, 'ALFKI', 71)]

In [5]:
# Gets all products which the customer have made and the product info
c.execute("""SELECT COUNT(*), CustomerALFKI.OrderID, CustomerALFKI.CustomerID FROM 
            ( SELECT * FROM Orders WHERE CustomerID = 'ALFKI' ) CustomerALFKI 
            INNER JOIN 'Order Details' od
            on CustomerALFKI.OrderID = od.OrderID
            INNER JOIN 'Products' pd
            on od.ProductID = pd.ProductID
            GROUP BY od.OrderID
            HAVING COUNT(*) > 1""")
c.fetchall()

[(3, 10643, 'ALFKI'),
 (2, 10702, 'ALFKI'),
 (2, 10835, 'ALFKI'),
 (2, 10952, 'ALFKI'),
 (2, 11011, 'ALFKI')]

In [6]:
# Setting up mongo client
client = MongoClient()
# Choosing db
db = client.Northwind
# Getting collections of interest
customers_collection = db['customers']
orders_collection = db['orders']
order_details_collection = db['order-details']
products_collection = db['products']

In [7]:
orders_products_pipeline_for_customer = [
                                            { "$match": { "CustomerID": "ALFKI" } },
                                            { "$lookup" : 
                                             {
                                                 "from" : "order-details",
                                                 "localField" : "OrderID",
                                                 "foreignField" : "OrderID",
                                                 "as" : "orders_for_customer"
                                             }
                                            },
                                            { "$lookup" : 
                                             {
                                                 "from" : "products",
                                                 "localField" : "orders_for_customer.ProductID",
                                                 "foreignField" : "ProductID",
                                                 "as" : "products_for_order"
                                             }
                                            }
                                        ]

In [8]:
pprint.pprint(list(orders_collection.aggregate(orders_products_pipeline_for_customer)))

[{'CustomerID': 'ALFKI',
  'EmployeeID': 6,
  'Freight': 29.46,
  'OrderDate': '1997-08-25 00:00:00.000',
  'OrderID': 10643,
  'RequiredDate': '1997-09-22 00:00:00.000',
  'ShipAddress': 'Obere Str. 57',
  'ShipCity': 'Berlin',
  'ShipCountry': 'Germany',
  'ShipName': 'Alfreds Futterkiste',
  'ShipPostalCode': 12209,
  'ShipRegion': 'NULL',
  'ShipVia': 1,
  'ShippedDate': '1997-09-02 00:00:00.000',
  '_id': ObjectId('59dccb9800e57dd96b616094'),
  'orders_for_customer': [{'Discount': 0.25,
                           'OrderID': 10643,
                           'ProductID': 28,
                           'Quantity': 15,
                           'UnitPrice': 45.6,
                           '_id': ObjectId('59dccb9800e57dd96b615aa7')},
                          {'Discount': 0.25,
                           'OrderID': 10643,
                           'ProductID': 39,
                           'Quantity': 21,
                           'UnitPrice': 18.0,
                           '_

In [9]:
uniq_orders_products_pipeline_for_customer = [
                                                { "$match": { "CustomerID": "ALFKI" } },
                                                { "$lookup" : 
                                                 {
                                                     "from" : "order-details",
                                                     "localField" : "OrderID",
                                                     "foreignField" : "OrderID",
                                                     "as" : "orders_for_customer"
                                                 }
                                                },
                                                { "$lookup" : 
                                                 {
                                                     "from" : "products",
                                                     "localField" : "orders_for_customer.ProductID",
                                                     "foreignField" : "ProductID",
                                                     "as" : "products_for_order"
                                                 }
                                                },
                                                { "$group" : 
                                                 {
                                                    "_id" : "$orders_for_customer.OrderID",
                                                 } 
                                                },
                                                { "$project" : 
                                                 {
                                                     "_id" : 1,
                                                     "number_of_orders" : { "$size" : "$_id" },
                                                 }
                                                },
                                                { "$match" : { "number_of_orders" : { "$gte" : 2 } } },
                                                { "$project" : 
                                                 {
                                                     "_id" : { "$setUnion" : [ "$_id"] },
                                                     "number_of_orders" : "$number_of_orders",
                                                 }
                                                }
                                            ]

In [10]:
pprint.pprint(list(orders_collection.aggregate(uniq_orders_products_pipeline_for_customer)))

[{'_id': [11011], 'number_of_orders': 2},
 {'_id': [10835], 'number_of_orders': 2},
 {'_id': [10952], 'number_of_orders': 2},
 {'_id': [10702], 'number_of_orders': 2},
 {'_id': [10643], 'number_of_orders': 3}]


In [11]:
# Getting customer ALFKI document
pprint.pprint(customers_collection.find_one({"CustomerID": "ALFKI"}))

{'Address': 'Obere Str. 57',
 'City': 'Berlin',
 'CompanyName': 'Alfreds Futterkiste',
 'ContactName': 'Maria Anders',
 'ContactTitle': 'Sales Representative',
 'Country': 'Germany',
 'CustomerID': 'ALFKI',
 'Fax': '030-0076545',
 'Phone': '030-0074321',
 'PostalCode': 12209,
 'Region': 'NULL',
 '_id': ObjectId('59dccb9700e57dd96b6148ed')}
